In [1]:
#configuración en google colab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
! pip install stargazer -q
! pip install linearmodels -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from stargazer.stargazer import Stargazer
from linearmodels.panel import PanelOLS, RandomEffects, PooledOLS

In [4]:
# directorios (path):
path = "gdrive/MyDrive/my_notebooks/datasets/"

In [5]:
# Cargar información con rezagos
df = pd.read_pickle(path + 'df_rezagos.pkl')
df

,Ano,Semana,Zona,Finca,municipio,Semana_seq,Ano_Semana,temp_media_lag1,temp_media_lag2,temp_media_lag3,...,Velocidad_del_viento_lag43,Velocidad_del_viento_lag44,Velocidad_del_viento_lag45,Velocidad_del_viento_lag46,Velocidad_del_viento_lag47,Velocidad_del_viento_lag48,Velocidad_del_viento_lag49,Velocidad_del_viento_lag50,Velocidad_del_viento_lag51,Peso_total_del_racimo_kg
0,2021,24,CENTRO,MADRE SELVA,CAREPA,1,202124,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.47
1,2021,25,CENTRO,MADRE SELVA,CAREPA,2,202125,27.89,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.78
2,2021,26,CENTRO,MADRE SELVA,CAREPA,3,202126,27.71,27.89,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.84
3,2021,27,CENTRO,MADRE SELVA,CAREPA,4,202127,26.44,27.71,27.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.81
4,2021,28,CENTRO,MADRE SELVA,CAREPA,5,202128,27.34,26.44,27.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2024,14,SUR,CORALES,CHIGORODÓ,147,202414,28.49,29.01,27.60,...,0.3,0.1,0.2,0.5,0.2,0.3,0.2,0.5,0.5,16.93
449,2024,15,SUR,CORALES,CHIGORODÓ,148,202415,28.18,28.49,29.01,...,0.2,0.3,0.1,0.2,0.5,0.2,0.3,0.2,0.5,16.75
450,2024,16,SUR,CORALES,CHIGORODÓ,149,202416,28.27,28.18,28.49,...,0.8,0.2,0.3,0.1,0.2,0.5,0.2,0.3,0.2,16.58
451,2024,17,SUR,CORALES,CHIGORODÓ,150,202417,27.87,28.27,28.18,...,0.4,0.8,0.2,0.3,0.1,0.2,0.5,0.2,0.3,16.83


In [6]:
# Generar un identificador por Zona
df.loc[df["Zona"] == "NORTE", "id"] = 1
df.loc[df["Zona"] == "CENTRO", "id"] = 2
df.loc[df["Zona"] == "SUR", "id"] = 3
df

,Ano,Semana,Zona,Finca,municipio,Semana_seq,Ano_Semana,temp_media_lag1,temp_media_lag2,temp_media_lag3,...,Velocidad_del_viento_lag44,Velocidad_del_viento_lag45,Velocidad_del_viento_lag46,Velocidad_del_viento_lag47,Velocidad_del_viento_lag48,Velocidad_del_viento_lag49,Velocidad_del_viento_lag50,Velocidad_del_viento_lag51,Peso_total_del_racimo_kg,id
0,2021,24,CENTRO,MADRE SELVA,CAREPA,1,202124,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.47,2.0
1,2021,25,CENTRO,MADRE SELVA,CAREPA,2,202125,27.89,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.78,2.0
2,2021,26,CENTRO,MADRE SELVA,CAREPA,3,202126,27.71,27.89,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.84,2.0
3,2021,27,CENTRO,MADRE SELVA,CAREPA,4,202127,26.44,27.71,27.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.81,2.0
4,2021,28,CENTRO,MADRE SELVA,CAREPA,5,202128,27.34,26.44,27.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.77,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2024,14,SUR,CORALES,CHIGORODÓ,147,202414,28.49,29.01,27.60,...,0.1,0.2,0.5,0.2,0.3,0.2,0.5,0.5,16.93,3.0
449,2024,15,SUR,CORALES,CHIGORODÓ,148,202415,28.18,28.49,29.01,...,0.3,0.1,0.2,0.5,0.2,0.3,0.2,0.5,16.75,3.0
450,2024,16,SUR,CORALES,CHIGORODÓ,149,202416,28.27,28.18,28.49,...,0.2,0.3,0.1,0.2,0.5,0.2,0.3,0.2,16.58,3.0
451,2024,17,SUR,CORALES,CHIGORODÓ,150,202417,27.87,28.27,28.18,...,0.8,0.2,0.3,0.1,0.2,0.5,0.2,0.3,16.83,3.0


Generar multi-indice con 'id' y 'Semana_seq', de forma que contenga una llave para ordenamiento de tiempo

In [7]:
df.set_index(["id", "Semana_seq"], inplace=True)
df

Ano  Semana    Zona        Finca  municipio  Ano_Semana  \
id  Semana_seq                                                             
2.0 1           2021      24  CENTRO  MADRE SELVA     CAREPA      202124   
    2           2021      25  CENTRO  MADRE SELVA     CAREPA      202125   
    3           2021      26  CENTRO  MADRE SELVA     CAREPA      202126   
    4           2021      27  CENTRO  MADRE SELVA     CAREPA      202127   
    5           2021      28  CENTRO  MADRE SELVA     CAREPA      202128   
...              ...     ...     ...          ...        ...         ...   
3.0 147         2024      14     SUR      CORALES  CHIGORODÓ      202414   
    148         2024      15     SUR      CORALES  CHIGORODÓ      202415   
    149         2024      16     SUR      CORALES  CHIGORODÓ      202416   
    150         2024      17     SUR      CORALES  CHIGORODÓ      202417   
    151         2024      18     SUR      CORALES  CHIGORODÓ      202418   

                temp_media_lag1  temp_media_lag2  temp_media_lag3  \
id  Semana_seq                                                      
2.0 1                       NaN              NaN              NaN   
    2                     27.89              NaN              NaN   
    3                     27.71            27.89              NaN   
    4                     26.44            27.71            27.89   
    5                     27.34            26.44            27.71   
...                         ...              ...              ...   
3.0 147                   28.49            29.01            27.60   
    148                   28.18            28.49            29.01   
    149                   28.27            28.18            28.49   
    150                   27.87            28.27            28.18   
    151                   28.52            27.87            28.27   

                temp_media_lag4  ...  Velocidad_del_viento_lag43  \
id  Semana_seq                   ...                               
2.0 1                       NaN  ...                         NaN   
    2                       NaN  ...                         NaN   
    3                       NaN  ...                         NaN   
    4                       NaN  ...                         NaN   
    5                     27.89  ...                         NaN   
...                         ...  ...                         ...   
3.0 147                   28.01  ...                         0.3   
    148                   27.60  ...                         0.2   
    149                   29.01  ...                         0.8   
    150                   28.49  ...                         0.4   
    151                   28.18  ...                         1.0   

                Velocidad_del_viento_lag44  Velocidad_del_viento_lag45  \
id  Semana_seq                                                           
2.0 1                                  NaN                         NaN   
    2                                  NaN                         NaN   
    3                                  NaN                         NaN   
    4                                  NaN                         NaN   
    5                                  NaN                         NaN   
...                                    ...                         ...   
3.0 147                                0.1                         0.2   
    148                                0.3                         0.1   
    149                                0.2                         0.3   
    150                                0.8                         0.2   
    151                                0.4                         0.8   

                Velocidad_del_viento_lag46  Velocidad_del_viento_lag47  \
id  Semana_seq                                                           
2.0 1                                  NaN                         NaN   
    2                                  NaN                  

## Realizar prueba de hausman para determinar tipos de efecto en el dataframe completo

In [8]:
def calculate_aic_bic_hqic(res):
  import numpy as np
  k = len(res.params)
  llf = res.loglik
  aic = 2 * k - 2 * llf
  bic = np.log(res.nobs) * k - 2 * llf
  hqic = 2 * k * np.log(np.log(res.nobs)) - 2 * llf
  return aic, bic, hqic

In [9]:
def hausman(fe, re):
  from scipy import stats
  b = fe.params
  B = re.params
  v_b = fe.cov
  v_B = re.cov
  df = b[np.abs(b) < 1e8].size
  chi2 = np.dot((b - B).T, np.linalg.inv(v_b - v_B).dot(b - B))
  pval = stats.chi2.sf(chi2, df)
  return chi2, df, pval

Crear dataframe para almacenar los resultados de los modelos para cada # de rezago

In [10]:
# lista de regresiones lineales
lista_modelos = pd.DataFrame(columns=['variables_predictoras', 'modelo', 'tipo_efectos'])
lista_modelos

,variables_predictoras,modelo,tipo_efectos


Crear lista con todas las variables rezagadas para realizar la selección entre ellas

In [11]:
variables_predictoras = []
for i in range(1, 52):  # Desde 1 hasta 51
  variables_predictoras.extend([f"temp_media_lag{i}", f"Precipitacion_lag{i}",
                                f"Humedad_relativa_lag{i}", f"Velocidad_del_viento_lag{i}"])

In [12]:
models = []  # Lista para almacenar los resultados del modelo

for i in range(1, 52):  # Desde 1 hasta 51
  exog_data = df[variables_predictoras[:i*4]]  # Seleccionar las primeras i*4 variables
  exog_data = sm.add_constant(exog_data)  # Agregar la constante

  # Fit the models
  #pool = PooledOLS(data.Peso_total_del_racimo_kg, exog_data)
  #pool_res = pool.fit()
  #print(pool_res)

  aleatorios = RandomEffects(df['Peso_total_del_racimo_kg'], exog_data)
  al_res = aleatorios.fit()

  fijos = PanelOLS(df['Peso_total_del_racimo_kg'], exog_data, entity_effects=True)
  fe_res = fijos.fit()

  aic_fe, bic_fe, hqic_fe = calculate_aic_bic_hqic(fe_res)
  aic_al, bic_al, hqic_al = calculate_aic_bic_hqic(al_res)

  hausman_results = hausman(fe_res, al_res)

  chi2 = hausman_results[0]
  pval = hausman_results[2]

  if chi2 < 0:
    print('chi2:', chi2)
    print("Efectos fijos son preferibles.")
    print(fe_res)
    lista_modelos.loc[len(lista_modelos)] = [variables_predictoras[:i*4],fe_res, 'Efectos fijos']
    rmse_fe = np.sqrt(np.mean(fe_res.resids ** 2))
    mae_fe = np.mean(np.abs(fe_res.resids))
    stargazer = Stargazer([fe_res])
    stargazer.add_line("RMSE", [f"{rmse_fe:.4f}"])
    stargazer.add_line("MAE", [f"{mae_fe:.4f}"])
    stargazer.add_line("AIC", [f"{aic_fe:.4f}"])
    stargazer.add_line("BIC", [f"{bic_fe:.4f}"])
    stargazer.add_line("HQIC", [f"{hqic_fe:.4f}"])
    models.append({"name": f"Modelo {i}", "result": fe_res, "aic": aic_al, "bic": bic_al, "hqic": hqic_al})

  elif pval < 0.05:
    print("Efectos fijos son preferibles (p-valor < 0.05).")
    print(fe_res)
    lista_modelos.loc[len(lista_modelos)] = [variables_predictoras[:i*4], fe_res, 'Efectos fijos']
    rmse_fe = np.sqrt(np.mean(fe_res.resids ** 2))
    mae_fe = np.mean(np.abs(fe_res.resids))
    stargazer = Stargazer([fe_res])
    stargazer.add_line("RMSE", [f"{rmse_fe:.4f}"])
    stargazer.add_line("MAE", [f"{mae_fe:.4f}"])
    stargazer.add_line("AIC", [f"{aic_fe:.4f}"])
    stargazer.add_line("BIC", [f"{bic_fe:.4f}"])
    stargazer.add_line("HQIC", [f"{hqic_fe:.4f}"])
    models.append({"name": f"Modelo {i}", "result": al_res, "aic": aic_al, "bic": bic_al, "hqic": hqic_al})
  else:
    print("Efectos aleatorios son preferibles.")
    print(al_res)
    lista_modelos.loc[len(lista_modelos)] = [variables_predictoras[:i*4], al_res, 'Efectos Aleatorios']
    rmse_al = np.sqrt(np.mean(al_res.resids ** 2))
    mae_al = np.mean(np.abs(al_res.resids))
    stargazer = Stargazer([al_res])
    stargazer.add_line("RMSE", [f"{rmse_al:.4f}"])
    stargazer.add_line("MAE", [f"{mae_al:.4f}"])
    stargazer.add_line("AIC", [f"{aic_al:.4f}"])
    stargazer.add_line("BIC", [f"{bic_al:.4f}"])
    stargazer.add_line("HQIC", [f"{hqic_al:.4f}"])
    models.append({"name": f"Modelo {i}", "result": al_res, "aic": aic_al, "bic": bic_al, "hqic": hqic_al})


/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:2623: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


chi2: -101.17072314953589
Efectos fijos son preferibles.
                             PanelOLS Estimation Summary                              
Dep. Variable:     Peso_total_del_racimo_kg   R-squared:                        0.0792
Estimator:                         PanelOLS   R-squared (Between):             -0.0181
No. Observations:                       450   R-squared (Within):               0.0792
Date:                      Fri, Jun 14 2024   R-squared (Overall):              0.0086
Time:                              23:10:04   Log-likelihood                   -1019.9
Cov. Estimator:                  Unadjusted                                           
                                              F-statistic:                      9.5206
Entities:                                 3   P-value                           0.0000
Avg Obs:                             150.00   Distribution:                   F(4,443)
Min Obs:                             150.00                              

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:2623: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


chi2: -118.30211582876387
Efectos fijos son preferibles.
                             PanelOLS Estimation Summary                              
Dep. Variable:     Peso_total_del_racimo_kg   R-squared:                        0.1160
Estimator:                         PanelOLS   R-squared (Between):             -0.0124
No. Observations:                       447   R-squared (Within):               0.1160
Date:                      Fri, Jun 14 2024   R-squared (Overall):              0.0230
Time:                              23:10:05   Log-likelihood                   -1005.4
Cov. Estimator:                  Unadjusted                                           
                                              F-statistic:                      7.1529
Entities:                                 3   P-value                           0.0000
Avg Obs:                             149.00   Distribution:                   F(8,436)
Min Obs:                             149.00                              

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:2623: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


...

chi2: -468.6896014254184
Efectos fijos son preferibles.
                             PanelOLS Estimation Summary                              
Dep. Variable:     Peso_total_del_racimo_kg   R-squared:                        0.9004
Estimator:                         PanelOLS   R-squared (Between):              0.3773
No. Observations:                       306   R-squared (Within):               0.9004
Date:                      Fri, Jun 14 2024   R-squared (Overall):              0.5102
Time:                              23:10:52   Log-likelihood                   -349.39
Cov. Estimator:                  Unadjusted                                           
                                              F-statistic:                      4.9379
Entities:                                 3   P-value                           0.0000
Avg Obs:                             102.00   Distribution:                 F(196,107)
Min Obs:                             102.00                               

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:2623: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


chi2: -1613.611573629349
Efectos fijos son preferibles.
                             PanelOLS Estimation Summary                              
Dep. Variable:     Peso_total_del_racimo_kg   R-squared:                        0.9060
Estimator:                         PanelOLS   R-squared (Between):              0.4124
No. Observations:                       303   R-squared (Within):               0.9060
Date:                      Fri, Jun 14 2024   R-squared (Overall):              0.5355
Time:                              23:10:53   Log-likelihood                   -335.39
Cov. Estimator:                  Unadjusted                                           
                                              F-statistic:                      4.8189
Entities:                                 3   P-value                           0.0000
Avg Obs:                             101.00   Distribution:                 F(200,100)
Min Obs:                             101.00                               

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:2623: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Efectos fijos son preferibles (p-valor < 0.05).
                             PanelOLS Estimation Summary                              
Dep. Variable:     Peso_total_del_racimo_kg   R-squared:                        0.9123
Estimator:                         PanelOLS   R-squared (Between):              0.4914
No. Observations:                       300   R-squared (Within):               0.9123
Date:                      Fri, Jun 14 2024   R-squared (Overall):              0.5945
Time:                              23:10:54   Log-likelihood                   -320.02
Cov. Estimator:                  Unadjusted                                           
                                              F-statistic:                      4.7405
Entities:                                 3   P-value                           0.0000
Avg Obs:                            100.000   Distribution:                  F(204,93)
Min Obs:                            100.000                                       

In [13]:
results = [model["result"] for model in models]
stargazer = Stargazer(results)

# Collect metrics for all models
rmse_values = [f"{np.sqrt(np.mean(model['result'].resids ** 2)):.4f}" for model in models]
mae_values = [f"{np.mean(np.abs(model['result'].resids)):.4f}" for model in models]
aic_values = [f"{model['aic']:.4f}" for model in models]
bic_values = [f"{model['bic']:.4f}" for model in models]
hqic_values = [f"{model['hqic']:.4f}" for model in models]

# Add lines to Stargazer with lists of values
stargazer.add_line("RMSE", rmse_values)
stargazer.add_line("MAE", mae_values)
stargazer.add_line("AIC", aic_values)
stargazer.add_line("BIC", bic_values)
stargazer.add_line("HQIC", hqic_values)

stargazer  # Display the table

In [14]:
lista_modelos['conteo_variables_predictoras'] = lista_modelos['variables_predictoras'].apply(lambda x: len(x))
lista_modelos['tipo_efectos'].value_counts()

tipo_efectos
Efectos fijos         48
Efectos Aleatorios     3
Name: count, dtype: int64

In [15]:
lista_modelos['tipo_efectos'].value_counts('%')

tipo_efectos
Efectos fijos         0.941176
Efectos Aleatorios    0.058824
Name: proportion, dtype: float64

In [16]:
lista_modelos

,variables_predictoras,modelo,tipo_efectos,conteo_variables_predictoras
0,"[temp_media_lag1, Precipitacion_lag1, Humedad_...",PanelOLS Estimati...,Efectos fijos,4
1,"[temp_media_lag1, Precipitacion_lag1, Humedad_...",PanelOLS Estimati...,Efectos fijos,8
2,"[temp_media_lag1, Precipitacion_lag1, Humedad_...",PanelOLS Estimati...,Efectos fijos,12
3,"[temp_media_lag1, Precipitacion_lag1, Humedad_...",PanelOLS Estimati...,Efectos fijos,16
4,"[temp_media_lag1, Precipitacion_lag1, Humedad_...",PanelOLS Estimati...,Efectos fijos,20
5,"[temp_media_lag1, Precipitacion_lag1, Humedad_...",PanelOLS Estimati...,Efectos fijos,24
6,"[temp_media_lag1, Precipitacion_lag1, Humedad_...",PanelOLS Estimati...,Efectos fijos,28
7,"[temp_media_lag1, Precipitacion_lag1, Humedad_...",PanelOLS Estimati...,Efectos fijos,32
8,"[temp_media_lag1, Precipitacion_lag1, Humedad_...",PanelOLS Estimati...,Efectos fijos,36
9,"[temp_media_lag1, Precipitacion_lag1, Humedad_...",PanelOLS Estimati...,Efectos fijos,40


Vemos que para el 94% de los modelos entrenados en funcion de los rezagos responden a un efecto fijo, por lo cual se agregaran las variables dummies por finca para que ahi se recoga dicho efecto en los interceptos